In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    "in_rolls_state_name_age_sex_fn_ln.csv.gz",
    usecols=["state", "birth_year", "first_name", "sex"],
)
df

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav
562652716,female,uttarakhand,1987.0,priyamkaa


In [3]:
# V2: Clean non-alpha from first_name
df["first_name"] = df.first_name.str.replace("[^a-zA-Z]", "", regex=True).str.strip()
df

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav
562652716,female,uttarakhand,1987.0,priyamkaa


In [4]:
df.dropna(subset=["first_name"], inplace=True)
df

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652712,female,uttarakhand,1959.0,sarooj
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav


* i think it makes sense to only release data where count for last_name/first_name is > 1000
* it will take out a bunch of weird names for now

In [5]:
adf = df[df["first_name"].groupby(df["first_name"]).transform("size") > 1000]
adf

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652712,female,uttarakhand,1959.0,sarooj
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav


In [6]:
adf["first_name"].groupby(adf["first_name"]).size()

first_name
             4244
aa          21192
aaaamsoo     2036
aaachal      2147
aaam         3673
            ...  
zeenat       1709
zeenath     12605
zohra        1934
zubaida      1263
zubair       1320
Name: first_name, Length: 40582, dtype: int64

In [7]:
adf = adf[adf.first_name.str.contains("[a-z]")]
adf = adf[~adf.first_name.str.contains(r"\.")]
adf

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652712,female,uttarakhand,1959.0,sarooj
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav


In [8]:
gdf = (
    adf.groupby(["state", "birth_year", "first_name"])
    .sex.value_counts()
    .unstack(fill_value=0)
)

In [9]:
gdf

sex                                  female  male  third gender
state       birth_year first_name                              
andaman     1902.0     samir              0     1             0
            1904.0     siri               0     1             0
            1905.0     cyril              0     1             0
            1908.0     shree              0     1             0
            1911.0     shanti             1     0             0
...                                     ...   ...           ...
uttarakhand 2016.0     syaam              0     1             0
                       ushaa              1     0             0
            2017.0     chandrakalee       1     0             0
                       kiranavathee       1     0             0
                       samjay             0     1             0

[10226249 rows x 3 columns]

In [10]:
gdf.reset_index(inplace=True)
gdf

sex,state,birth_year,first_name,female,male,third gender
0,andaman,1902.0,samir,0,1,0
1,andaman,1904.0,siri,0,1,0
2,andaman,1905.0,cyril,0,1,0
3,andaman,1908.0,shree,0,1,0
4,andaman,1911.0,shanti,1,0,0
...,...,...,...,...,...,...
10226244,uttarakhand,2016.0,syaam,0,1,0
10226245,uttarakhand,2016.0,ushaa,1,0,0
10226246,uttarakhand,2017.0,chandrakalee,1,0,0
10226247,uttarakhand,2017.0,kiranavathee,1,0,0


In [11]:
gdf.columns = [
    "state",
    "birth_year",
    "first_name",
    "n_female",
    "n_male",
    "n_third_gender",
]
gdf

,state,birth_year,first_name,n_female,n_male,n_third_gender
0,andaman,1902.0,samir,0,1,0
1,andaman,1904.0,siri,0,1,0
2,andaman,1905.0,cyril,0,1,0
3,andaman,1908.0,shree,0,1,0
4,andaman,1911.0,shanti,1,0,0
...,...,...,...,...,...,...
10226244,uttarakhand,2016.0,syaam,0,1,0
10226245,uttarakhand,2016.0,ushaa,1,0,0
10226246,uttarakhand,2017.0,chandrakalee,1,0,0
10226247,uttarakhand,2017.0,kiranavathee,1,0,0


In [12]:
gdf["prop_female"] = gdf["n_female"] / (
    gdf["n_female"] + gdf["n_male"] + gdf["n_third_gender"]
)
gdf

,state,birth_year,first_name,n_female,n_male,n_third_gender,prop_female
0,andaman,1902.0,samir,0,1,0,0.0
1,andaman,1904.0,siri,0,1,0,0.0
2,andaman,1905.0,cyril,0,1,0,0.0
3,andaman,1908.0,shree,0,1,0,0.0
4,andaman,1911.0,shanti,1,0,0,1.0
...,...,...,...,...,...,...,...
10226244,uttarakhand,2016.0,syaam,0,1,0,0.0
10226245,uttarakhand,2016.0,ushaa,1,0,0,1.0
10226246,uttarakhand,2017.0,chandrakalee,1,0,0,1.0
10226247,uttarakhand,2017.0,kiranavathee,1,0,0,1.0


In [13]:
gdf.to_csv("in_rolls_state_year_fn_naampy.csv.gz", index=False, compression="gzip")

In [14]:
bdf = (
    gdf.groupby(["first_name"])
    .agg({"n_female": "sum", "n_male": "sum", "n_third_gender": "sum"})
    .reset_index()
)
bdf["prop_female"] = bdf["n_female"] / (
    bdf["n_female"] + bdf["n_male"] + gdf["n_third_gender"]
)
bdf

,first_name,n_female,n_male,n_third_gender,prop_female
0,aa,948,20244,0,0.044734
1,aaaamsoo,1375,661,0,0.675344
2,aaachal,2092,55,0,0.974383
3,aaam,135,3538,0,0.036755
4,aaambaath,880,737,0,0.544218
...,...,...,...,...,...
40576,zeenat,1703,6,0,0.996489
40577,zeenath,12583,22,0,0.998255
40578,zohra,1931,3,0,0.998449
40579,zubaida,1260,3,0,0.997625


In [1]:
# Resource List: procs=1,mem=200gb
# Resources Used: cput=00:36:25,vmem=130413968kb,walltime=00:37:04,mem=134660996kb,energy_used=0